In [5]:
import timeit
import numpy as np
from time import time

In [4]:
from numba import cuda
import numba as nb
import math

In [ ]:
print(cuda.gpus)

In [ ]:
cuda.select_device(0)

In [ ]:
# %%time
# Initialize the data arrays
A = np.asfortranarray(np.random.randn(32*2, 32*3))
B = np.asfortranarray(np.random.randn(3*32, 32))
C = np.asfortranarray(np.random.randn(32*2, 32))

In [ ]:
%%time
# Copy the arrays to the device
A_global_mem = cuda.to_device(A)
B_global_mem = cuda.to_device(B)
C_global_mem = cuda.to_device(C)

# Allocate memory on the device for the result
# C_global_mem = cuda.device_array((32*2, 32))

In [ ]:
%%time
# Configure the blocks
threadsperblock = (16,16)
blockspergrid_x = int(math.ceil(A.shape[0] / threadsperblock[0]))
blockspergrid_y = int(math.ceil(B.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

In [ ]:
blockspergrid

In [ ]:
(A.nbytes+B.nbytes+np.dot(A,B).nbytes)/1024/1024

In [ ]:
# CUDA kernel
@cuda.jit
def matmul(A, B, C):
    """Perform matrix multiplication of C = A * B
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp

In [ ]:
%%time
matmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)

In [ ]:
%%time
C = C_global_mem.copy_to_host()
print(C.shape)

In [ ]:
%%time
np.allclose(C,np.dot(A,B))

In [ ]:
np.dot(A,B)

In [ ]:
C

In [ ]:
# (524288, 128)

In [13]:
%time
@cuda.jit
def my_kernel_2D(io_array,bs):
    x, y = cuda.grid(2)
    if x<io_array.shape[0] and y<io_array.shape[1]:
        io_array[x,y]=bs[y]

bs=np.arange(10)
data = np.ones((16, 10))
data_glob=cuda.to_device(data)
threadsperblock = (32, 32)
blockspergrid_x = math.ceil(data.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(data.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)
my_kernel_2D[blockspergrid, threadsperblock](data_glob,bs)
print(data_glob.copy_to_host())

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.01 µs
[[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]]


In [ ]:
A.shape,B.shape

In [ ]:
TPB=16
@cuda.jit
def fast_matmul(A,B,C):
    sA=cuda.shared.array(shape=(TPB,TPB),dtype=nb.float32)
    sB=cuda.shared.array(shape=(TPB,TPB),dtype=nb.float32)
    x,y=cuda.grid(2)
    tx=cuda.threadIdx.x
    ty=cuda.threadIdx.y
    if x>=C.shape[0] and y>=C.shape[1]:
        return
    tmp=0
    for i in range(A.shape[1]//TPB):  # for number of blocks
        sA[tx,ty]=A[x,ty+i*TPB]       # preLoad data into shared memory
        sB[tx,ty]=B[tx+i*TPB,y]
        
        cuda.syncthreads()            # wait for loading
        for j in range(TPB):
            tmp+=sA[tx,j]*sB[j,ty]
            cuda.syncthreads()            # wait for computation
    C[x,y]=tmp

In [ ]:
%%time
fast_matmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)

In [ ]:
%%time
C = C_global_mem.copy_to_host()
print(C.shape)

In [ ]:
C

In [6]:
import pycuda

In [4]:
cudlib=ctypes.CDLL('libcublas.so')

In [5]:
N=128

In [ ]:
A = np.array(np.arange(N ** 2, dtype=np.float32).reshape(N, N), order='F')
B = np.array(np.arange(N) + 10, dtype=A.dtype, order='F')
D = np.zeros_like(A, order='F')

In [ ]:
np.asfortranarray(A).ctypes.data

In [ ]:
D

In [ ]:
cudlib.cublasSdot

In [ ]:
A_global_mem.device_ctypes_pointer

In [ ]:
cudlib.cublasSgemm('N', 'N', N, N, N, 1, A_global_mem.device_ctypes_pointer, B_global_mem.device_ctypes_pointer, 1, C_global_mem.device_ctypes_pointer)

In [1]:
import ctypes
import numpy as np

In [2]:
def init_kernel_bias(num_inp_channels, kernel_size, num_kernels,mean=0,std=0.01):
    shape = [num_inp_channels, kernel_size, kernel_size, num_kernels]
    weights = std*np.random.randn(*shape) + mean
    # weights/=np.sqrt(num_inp_channels)
    bias = std*np.random.randn(1,num_kernels) + mean
    return weights.astype(np.float32), bias.astype(np.float32)

In [3]:
w0,b0=init_kernel_bias(num_inp_channels=32,kernel_size=3,num_kernels=128)

In [4]:
inp=np.random.randn(128,64,64,32).astype(np.float32)

In [5]:
#inp[batches,row,col,d],w0(d,ksz,ksz,num_ker),b0[1,num_ker],stride[row,col]
padding=0
stride=[1,1]
ipp=inp.transpose(0,3,1,2)  #ipp[batches,d,row,col]
output=[]
ksz=w0.shape[1]
num_ker=w0.shape[3]
if not padding: #take care of padding in backprop too
    padding=(ksz-1)//2  #currently don't give 'even' ksz
out_row,out_col=((ipp.shape[2]-ksz+2*padding)//stride[0]+1),((ipp.shape[3]-ksz+2*padding)//stride[1]+1)
batches,d,row,col=ipp.shape
row+=2*padding
col+=2*padding
padded=np.zeros((batches,d,row,col)).astype(np.float32)
padded[:,:,padding:-padding,padding:-padding]=ipp

In [6]:
img=padded[0]

In [7]:
# %%timeit
window=(np.arange(ksz)[:,None]*row+np.arange(ksz)).ravel()+np.arange(d)[:,None]*row*col
slider=(np.arange(out_row*stride[0])[:,None]*row+np.arange(out_col*stride[1]))
ind = window.ravel()+slider[::stride[0],::stride[1]].ravel()[:,None]
# bind= np.arange(batches)[:,None]*d*row*col+ind.ravel()
kern = w0.reshape(-1,num_ker)
# output=(np.dot(np.take(padded, bind).reshape(-1,d*ksz*ksz), kern)).reshape(batches,out_row,out_col,num_ker)

In [8]:
%%time
output=np.empty((batches,out_row*out_col,num_ker),dtype=np.float32)
for i,img in enumerate(padded):      #img[d,row,col]
    # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
    output[i]=np.dot(img.take(ind), kern)
# output+=b0
ans2=output.reshape(batches,out_row,out_col,num_ker)

CPU times: user 2.66 s, sys: 31.9 ms, total: 2.69 s
Wall time: 688 ms


In [9]:
%%time
coled=np.empty((batches,*ind.shape),dtype=np.float32)#,order='F')
for i,img in enumerate(padded):      #img[d,row,col]
    # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
    coled[i]=img.take(ind)
# output+=b0
# ans2=output.reshape(batches,out_row,out_col,num_ker)
# output=(np.dot(np.take(padded, bind), self.kern)+self.biases)
# print(coled.nbytes/1024/1024)

CPU times: user 317 ms, sys: 36.4 ms, total: 353 ms
Wall time: 316 ms


In [10]:
%%time
a=coled.reshape(-1,d*ksz*ksz)
b=kern.reshape(d*ksz*ksz,-1)
(a.nbytes+b.nbytes)/1024/1024

CPU times: user 15 µs, sys: 0 ns, total: 15 µs
Wall time: 19.1 µs


In [11]:
c=np.empty((a.shape[0],b.shape[1]),dtype=np.float32,order='F')
c.nbytes/1024/1024

256.0

In [12]:
%%time
a=np.asfortranarray(a)
b=np.asfortranarray(b)
c=np.asfortranarray(c)

CPU times: user 1.19 s, sys: 33.3 ms, total: 1.23 s
Wall time: 1.23 s


In [13]:
np.isfortran(c)

True

In [14]:
c

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [15]:
sgemm=ctypes.CDLL('libsgemm.so')

In [16]:
al,bet=1,1

In [17]:
%%time
sgemm.gemm(ctypes.c_void_p(a.ctypes.data),ctypes.c_void_p(b.ctypes.data),ctypes.c_void_p(c.ctypes.data),a.shape[0],b.shape[0],c.shape[1],ctypes.c_float(al),ctypes.c_float(bet),ctypes.c_void_p(b0[0].ctypes.data))

CPU times: user 570 ms, sys: 334 ms, total: 904 ms
Wall time: 905 ms


0

In [18]:
a.shape[0],b.shape[0],c.shape[1]

(524288, 288, 128)

In [19]:
c.shape

(524288, 128)

In [20]:
# al*np.dot(a,b)+bet*c

In [21]:
# %timeit c+b0

In [36]:
c

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [30]:
sgemm.test(ctypes.c_void_p(b0[0].ctypes.data))

0

In [23]:
ans2.reshape(-1,128)

array([[ 0.19888292,  0.05908064, -0.1041484 , ...,  0.05050119,
        -0.02117762,  0.03724441],
       [ 0.13932133, -0.00317683, -0.1091186 , ..., -0.15248571,
        -0.00138388, -0.01931977],
       [ 0.24624307, -0.1992316 , -0.29994118, ...,  0.18592387,
        -0.01929658,  0.01150278],
       ...,
       [ 0.21800207,  0.06780856, -0.22162549, ...,  0.25347805,
        -0.05794849, -0.16329093],
       [ 0.01741691,  0.33883536,  0.03972958, ...,  0.09926359,
        -0.31703758,  0.06678851],
       [ 0.00752398, -0.10625924, -0.12107888, ...,  0.11845581,
         0.08636219,  0.09377596]], dtype=float32)

In [24]:
(c==ans2.reshape(-1,128)).all()

True

In [28]:
a=(np.arange(30)+11).reshape(6,5,order='F').astype(np.float32)  #mxk

In [29]:
b=(np.arange(20)+11).reshape(5,4,order='F').astype(np.float32)  #kxn

In [30]:
c=(np.arange(24)*0).reshape(6,4,order='F').astype(np.float32)  #mxn